# Generating Video Chapters

This notebook demonstrates how to automatically generate video chapters using Azure Content Understanding. By analyzing the video content, the LLM model applies its own reasoning to segment the video into meaningful chapters and scenes, providing concise descriptions and timestamps. This approach enables users to quickly create a structured table of contents for any video, powered by advanced AI understanding.


## Pre-requisites
1. Follow [README](../README.md#configure-azure-ai-service-resource) to create essential resource that will be used in this sample.
1. Install required packages

In [ ]:
%pip install -r ../requirements.txt

## Load environment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env", override=True)

AZURE_AI_SERVICE_ENDPOINT = os.getenv("AZURE_AI_SERVICE_ENDPOINT")
AZURE_AI_SERVICE_API_VERSION = os.getenv("AZURE_AI_SERVICE_API_VERSION", "2025-05-01-preview")

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

If you haven't done so, please authenticate by running **'az login'** through the terminal. This credentials are used to validate that you have access to the resources you defined above.

Make sure you have Azure CLI installed on your system. To install --> curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

In [ ]:

## Authehticate if you are running this notebook for the first time.

import subprocess

subprocess.run("az login", shell=True)


## File to Analyze

Use the following variable to define what file to analyze. For this example, we will be examining a small tutorial. If you would like to try with your own videos, you could use a small video to get results faster.

In [ ]:
VIDEO_LOCATION = "https://github.com/Azure-Samples/azure-ai-content-understanding-assets/raw/refs/heads/main/videos/learning/learning2.mp4"

# Automated Chapter Discovery

We will first use Content Understanding and let service interact with an LLM model to apply its own reasoning to segment the video into meaningful chapters, providing concise descriptions and timestamps.

## Create a Custom Analyzer
The custom analyzer schema for this notebook is [video_chapters_dynamic.json](../analyzer_templates/video_chapters_dynamic.json). This file defines the schema and configuration for a custom video analyzer that uses AI to dynamically generate chapters and scenes based on its understanding of the video content. The service analyzes the video and determines the most meaningful chapter divisions, providing start and end timestamps for each. The configuration section allows flexible segmentation, enabling the model to reason about the content and organize chapters in a way that best fits.

In this example, we will use the utility class `AzureContentUnderstandingClient` to load the analyzer schema from the template file and submit it to Azure Content Understanding service. Then, we will analyze the video and generate the desired chapter and scene structure.


In [ ]:
import sys
from pathlib import Path
import json
import uuid


# add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(
    str(parent_dir)
)
from python.content_understanding_client import AzureContentUnderstandingClient

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

# The analyzer template is used to define the schema of the output
ANALYZER_TEMPLATE_PATH = "../analyzer_templates/video_chapters_dynamic.json"
ANALYZER_ID = "video_scene_chapter" + "_" + str(uuid.uuid4())  # Unique identifier for the analyzer

# Create the Content Understanding (CU) client
cu_client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_SERVICE_ENDPOINT,
    api_version=AZURE_AI_SERVICE_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/video_chapters_dynamic", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

# Use the client to create an analyzer
response = cu_client.begin_create_analyzer(
    ANALYZER_ID, analyzer_template_path=ANALYZER_TEMPLATE_PATH)
result = cu_client.poll_result(response)

print(json.dumps(result, indent=2))

### Use the created analyzer to extract video content
With the analyzer created, request video content extraction using content understanding.

In [ ]:
# Submit the video for content analysis
response = cu_client.begin_analyze(ANALYZER_ID, file_location=VIDEO_LOCATION)

# Wait for the analysis to complete and get the content analysis result
video_cu_result = cu_client.poll_result(
    response, timeout_seconds=3600)  # 1 hour timeout for long videos

# Print the content analysis result
print(f"Video Content Understanding result: ", video_cu_result)

# Optional - Delete the analyzer if it is no longer needed
cu_client.delete_analyzer(ANALYZER_ID)

Use the following utility to display results for visual review.

In [ ]:
from IPython.display import display, HTML
from python.chapters_utility import ChaptersFormatter

full_html = ChaptersFormatter.format_chapters_output(VIDEO_LOCATION, video_cu_result)
display(HTML(full_html))

# Structured Chapter Creation

Alternatively, you can define a specific structure for chaptering that Content Understanding will use to guide the model. When your request includes information detailing clear chapter types and organization, the service can instruct the model to segment the video according to your desired structure, ensuring consistent and predictable chapter creation.

## Create a Custom Analyzer
The custom analyzer schema for this notebook is [video_chapters_structured.json](../analyzer_templates/video_chapters_structured.json). This file defines the schema and configuration for a custom video analyzer. In this example, it specifies how a video should be segmented into chapters and scenes, including three chapter types: "Topic Introduction", "Details About the Work Done", and "Conclusion or Results". Each segment contains a list of scenes, with each scene described by a short description, start timestamp, and end timestamp. The configuration section controls segmentation behavior and other analysis options, while the fieldSchema section outlines the expected structure of the output, ensuring chapters and scenes are clearly organized and non-overlapping.

We we will also be using the utility class `AzureContentUnderstandingClient` to load the analyzer schema from the template file and submit it to Azure Content Understanding service. Then, we will analyze the video and generate the desired chapter and scene structure.

In [ ]:
import sys
from pathlib import Path
import json
import uuid


# add the parent directory to the path to use shared modules
parent_dir = Path(Path.cwd()).parent
sys.path.append(
    str(parent_dir)
)
from python.content_understanding_client import AzureContentUnderstandingClient

from azure.identity import DefaultAzureCredential, get_bearer_token_provider
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

# The analyzer template is used to define the schema of the output
ANALYZER_TEMPLATE_PATH = "../analyzer_templates/video_chapters_structured.json"
ANALYZER_ID = "video_scene_chapter" + "_" + str(uuid.uuid4())  # Unique identifier for the analyzer

# Create the Content Understanding (CU) client
cu_client = AzureContentUnderstandingClient(
    endpoint=AZURE_AI_SERVICE_ENDPOINT,
    api_version=AZURE_AI_SERVICE_API_VERSION,
    token_provider=token_provider,
    x_ms_useragent="azure-ai-content-understanding-python/video_chapters_structured", # This header is used for sample usage telemetry, please comment out this line if you want to opt out.
)

# Use the client to create an analyzer
response = cu_client.begin_create_analyzer(
    ANALYZER_ID, analyzer_template_path=ANALYZER_TEMPLATE_PATH)
result = cu_client.poll_result(response)

print(json.dumps(result, indent=2))

### Use the created analyzer to extract video content
It might take some time depending on the video length. Try with short videos to get results faster

In [ ]:
# Submit the video for content analysis
response = cu_client.begin_analyze(ANALYZER_ID, file_location=VIDEO_LOCATION)

# Wait for the analysis to complete and get the content analysis result
video_cu_result = cu_client.poll_result(
    response, timeout_seconds=3600)  # 1 hour timeout for long videos

# Print the content analysis result
print(f"Video Content Understanding result: ", video_cu_result)

# Optional - Delete the analyzer if it is no longer needed
cu_client.delete_analyzer(ANALYZER_ID)

Use the following utility to display results for visual review.

In [ ]:
from IPython.display import display, HTML
from python.chapters_utility import ChaptersFormatter

full_html = ChaptersFormatter.format_chapters_output(VIDEO_LOCATION, video_cu_result)
display(HTML(full_html))

# Conclusion

This notebook has demonstrated how Azure Content Understanding, combined with advanced language models, can automatically generate meaningful chapters and scenes from video content. By leveraging AI-driven analysis, you can quickly create structured, searchable tables of contents for any video, making it easier to navigate and understand complex material. This approach streamlines video processing and enables new possibilities for content organization and discovery.